In [1]:
#유방암 데이터셋 나누기
import pandas as pd
df= pd.read_csv('https://archive.ics.uci.edu/ml/'
               'machine-learning-databases'
               '/breast-cancer-wisconsin/wdbc.data',
               header=None)



In [6]:
from sklearn.preprocessing import LabelEncoder
df.head

<bound method NDFrame.head of            0  1      2      3       4       5        6        7        8   \
0      842302  M  17.99  10.38  122.80  1001.0  0.11840  0.27760  0.30010   
1      842517  M  20.57  17.77  132.90  1326.0  0.08474  0.07864  0.08690   
2    84300903  M  19.69  21.25  130.00  1203.0  0.10960  0.15990  0.19740   
3    84348301  M  11.42  20.38   77.58   386.1  0.14250  0.28390  0.24140   
4    84358402  M  20.29  14.34  135.10  1297.0  0.10030  0.13280  0.19800   
..        ... ..    ...    ...     ...     ...      ...      ...      ...   
564    926424  M  21.56  22.39  142.00  1479.0  0.11100  0.11590  0.24390   
565    926682  M  20.13  28.25  131.20  1261.0  0.09780  0.10340  0.14400   
566    926954  M  16.60  28.08  108.30   858.1  0.08455  0.10230  0.09251   
567    927241  M  20.60  29.33  140.10  1265.0  0.11780  0.27700  0.35140   
568     92751  B   7.76  24.54   47.92   181.0  0.05263  0.04362  0.00000   

          9   ...      22     23      24     

In [9]:
X = df.loc[:,2:].values
y = df.loc[:, 1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [10]:
le.transform(['M','B'])

array([1, 0])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size=0.20,
                                                   stratify = y,
                                                   random_state=1)

In [14]:
#파이프 라인으로 변환기와 추정기 연결
#PCA를 사용하여 30차원에서 2차원으로 데이터 압축
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
pipe_lr = make_pipeline(StandardScaler(),
                       PCA(n_components =2),
                       LogisticRegression(solver='liblinear',random_state=1))
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
print('테스트 정확도: %.3f' % pipe_lr.score(X_test, y_test))

테스트 정확도: 0.956


### k겹 교차 검증을 사용한 모델 성능
홀드아웃방법   
원본 데이터셋을 훈련세트, 검증세트, 테스트 세트로 나누는 것

k-겹 교차 검증    
중복을 허락하지 않고 훈련 데이터셋을 k개의 폴드로 랜덤하게 나눔   
k-1개의 폴드로 모델을 훈련하고 나머지 하나의 폴드로 성능을 평가, k번 반복

계층적 k-겹 교차 검증

In [20]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(estimator = pipe_lr,
                        X=X_train,
                        y=y_train,
                        cv=10,
                        n_jobs=1)
print('CV 정확도 점수 : %s' %scores)

print('CV 정확도 : %.3f +/- %.3f' %(np.mean(scores),
                              np.std(scores)))


CV 정확도 점수 : [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도 : 0.950 +/- 0.014


In [21]:
###추후 추가 학습 예정